# Diffractometer Extra Motors and/or Pseudos

Sometimes, it is desired to add additional ophyd components to
a diffractometer object.  Such components could include additional motor axes, azimuthal reference vectors, temperature, etc.

**Objective**

Add one or more real positioners to the standard positioners of the 2-circle
diffractometer (*tth_tth*, *TH TTH Q* [geometry](../diffractometers.rst)).  Use
simulated motors for the example (no EPICS required).

## Standard 2-circle

First, we start with the setup of a 2-circle ($\theta:2\theta$) diffractometer.

axis | space
--- | ---
th | real
tth | real
q | pseudo

Create the diffractometer object.

In [1]:
import hklpy2

th2th = hklpy2.creator(name="th2th", geometry="TH TTH Q", solver="th_tth")

Show the diffractometer configuration and all the ophyd components.

In [2]:
th2th.wh()
print(f"{th2th.component_names=}")

wavelength=1.0
q=0
th=0, tth=0
th2th.component_names=('solver_signature', 'beam', 'q', 'th', 'tth')


## Add additional positioner

We can use the `hklpy2.creator()` for the additional positioner.  Since we are not using the default `reals`, we'll provide a Python dictionary that defines each real axis, in order, and whether it uses a simulated motor or an EPICS PV.  `None` means to use a simulated motor.

In [3]:
th2th = hklpy2.creator(
    name="th2th",
    geometry="TH TTH Q",
    solver="th_tth",
    reals=dict(th=None, tth=None, spinner=None)
)
th2th.wh()
print(f"{th2th.component_names=}")

wavelength=1.0
q=0
th=0, tth=0
th2th.component_names=('solver_signature', 'beam', 'q', 'th', 'tth', 'spinner')


Compare these results.  The new result adds the `spinner` axis.

Set (and show) the limits on the spinner:

In [4]:
th2th.spinner._limits = -10_000, 10_000
th2th.spinner.limits

(-10000, 10000)

## Can we add other pseudo axes?

**Q**: With this capability to add additional real positioners, can we add axes
to the pseudo positioners?

**A**: Yes. See this example.  It defines two pseudo axes: `Q` and `d`.  As shown in `aliases`, the `Q` axis is mapped to the {ref}`solver <geometries-th_tth-th-tth-q>` `q` axis.

In [5]:
th2th = hklpy2.creator(
    name="th2th",
    geometry="TH TTH Q",
    solver="th_tth",
    pseudos=["Q", "d"],  # 
    reals=dict(sample=None, detector=None, spinner=None),
    aliases=dict(pseudos=["Q"], reals=["sample", "detector"]),
)
th2th.wh()

wavelength=1.0
Q=0
sample=0, detector=0


## Add additional Signals and Devices

Finally, we add additional Signals and Component Devices as a demonstration.

The {func}function]`~hklpy2.diffract.creator()` has its limits.  The `creator()`
relies on a {func}function]`~hklpy2.diffract.diffractometer_class_factory()`.
Let's skip the factory function and show how to build a structure directly.

Demonstrate a variety of additional components.

In [6]:
from hklpy2.diffract import Hklpy2PseudoAxis
from ophyd import Component, Device, Signal, SoftPositioner
from ophyd.signal import SignalRO


class XYStage(Device):
    x = Component(SoftPositioner, kind="hinted", limits=(-20, 105), init_pos=0)
    y = Component(SoftPositioner, kind="hinted", limits=(-20, 105), init_pos=0)
    solenoid_lock = Component(Signal, value=True, kind="normal")


class MyTwoCircle(hklpy2.DiffractometerBase):
    _real = ["th", "tth"]

    q = Component(Hklpy2PseudoAxis, "", kind="hinted")
    th = Component(
        SoftPositioner, kind="hinted", limits=(-180, 180), egu="degrees", init_pos=0
    )
    tth = Component(
        SoftPositioner, kind="hinted", limits=(-180, 180), egu="degrees", init_pos=0
    )
    spinner = Component(
        SoftPositioner,
        kind="hinted",
        limits=(-10000, 10000),
        egu="rotations",
        init_pos=0,
    )
    atth = Component(
        SoftPositioner, kind="hinted", limits=(-180, 180), egu="degrees", init_pos=0
    )
    temperature = Component(SignalRO, value=25, kind="normal")
    xy = Component(XYStage, kind="normal")

    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            solver="th_tth",  # solver name
            geometry="TH TTH Q",  # solver geometry
            **kwargs,
        )


th2tth = MyTwoCircle(name="th2tth")
th2tth.wh()  # brief report of diffractometer position
# th2th.summary()  # show the full ophyd structure summary
th2tth.read()

wavelength=1.0
q=0
th=0, tth=0


OrderedDict([('th2tth_beam_wavelength',
              {'value': 1.0, 'timestamp': 1744256777.7262537}),
             ('th2tth_beam_energy',
              {'value': 12.398419843856837, 'timestamp': 1744256777.7262843}),
             ('th2tth_q', {'value': 0, 'timestamp': 1744256777.7263556}),
             ('th2tth_q_setpoint',
              {'value': 0, 'timestamp': 1744256777.726387}),
             ('th2tth_th', {'value': 0, 'timestamp': 1744256777.7611496}),
             ('th2tth_tth', {'value': 0, 'timestamp': 1744256777.7611551}),
             ('th2tth_spinner', {'value': 0, 'timestamp': 1744256777.7611592}),
             ('th2tth_atth', {'value': 0, 'timestamp': 1744256777.7611616}),
             ('th2tth_temperature',
              {'value': 25, 'timestamp': 1744256777.726868}),
             ('th2tth_xy_x', {'value': 0, 'timestamp': 1744256777.7611754}),
             ('th2tth_xy_y', {'value': 0, 'timestamp': 1744256777.7611783}),
             ('th2tth_xy_solenoid_lock',
          